In [4]:
import pandas as pd
import pandas_ta as ta
import datetime
import numpy as np
import pickle

In [66]:
import warnings
warnings.filterwarnings('ignore')

def connectToBinance():
  from binance import Client
  return Client('3FqLXkx5UOAX10B5lVRhmDDFap7uFY8evLeOqFqnwVfmkZh9IY1tr1to8oBwNdRG',
                'UUR5wLkgzqXEMQbRzkwxJlwD9iChahjLyYNanzawTCDX1jIxRUzpxVtZvcSyjAi1')

def getLast48Candles(client, symbol): #Close time, open, high, low, close, volume
  s = symbol.split('USDT')[0]

  tickers = client.get_historical_klines(symbol, '1h', limit=48)
  candles = []
  for ticker in tickers:
    candle = [np.float32(ticker[1]),np.float32(ticker[2]),np.float32(ticker[3]),np.float32(ticker[4]),np.float32(ticker[5]),ticker[6]]
    candles.append(candle)
  
  df = pd.DataFrame(candles, columns=['open', 'high', 'low','close', 'Volume '+ s, 'date'])
  
  df['date'] = pd.to_datetime(df['date'],unit='ms')
  df.set_index('date', inplace=True)
  return df


  
def getIndicators(data):
  strat = ta.Strategy(name='Strategie',
                     description='MACD, RSI',
                     ta=[
                      {'kind':'rsi', 'length':7},
                      {'kind':'rsi', 'length':14},
                      {'kind':'sma', 'length':7},
                      {'kind':'sma', 'length':25},
                      {'kind':'macd', 'fast':12 , 'slow':26},
                      {'kind':'obv'},
                     ])

  data.ta.strategy(strat)
  return data.iloc[-2]

def getPrediction(indicators, symbol):
  filename='Models/'+symbol+'_model.pkl'
  with open(filename, 'rb') as file:
    model = pickle.load(file)
  return model.predict(np.array(indicators).reshape(1, -1)), model.predict_proba(np.array(indicators).reshape(1, -1))
  
def sellPapier(amount, file):
  print(f"Une vente de {amount}BTC à été effectuée.\n\n")
  
def buyPapier(amount, file):
  print(f"Un achat de {amount}BTC à été effectué.\n\n")

def waitPapier(file):
  print("Aucune action à été effectué.\n\n")

def paperTrade(prediction, amount):
  file = open('log.txt', 'a')
  file.write(f"{datetime.datetime.now()} - La predicton {'UP' if prediction == 2 else ('DOWN' if prediction == 0 else 'NO_MOVEMENT')} a été émise.\n")
  
  
  if(prediction == 0):
    sellPapier(amount,file)
  if(prediction == 2):
    buyPapier(amount,file)
  if(prediction == 1):
    waitPapier(amount,file)
  
   
  

client = connectToBinance()
for symbol in ['ADAUSDT', 'BNBUSDT','BTCUSDT','ETHUSDT','LTCUSDT','XRPUSDT']:
  candles = getLast48Candles(client, symbol)
  indicators = getIndicators(candles)
  prediction, proba = getPrediction(indicators, symbol)
  print(symbol,prediction, proba[0][int(prediction)])


# 0:Down
# 1:no mouvement greater then +/-50$
# 2:Up
# import random
# prediction = random.randint(0, 2) ##Evidement à effacer
# amount = getAmount(client)
# paperTrade(prediction, amount)





# if(prediction == 0):
# elif(prediction==1):
# else:
  


ADAUSDT [0.] 0.501945430439115
BNBUSDT [1.] 0.512637682874826
BTCUSDT [1.] 0.5031273280967975
ETHUSDT [0.] 0.5021434912961291
LTCUSDT [1.] 0.5016437837018008
XRPUSDT [0.] 0.5236675122902797


In [35]:
#Log prediction
import datetime
import random

prediction = random.randint(0, 2)
f = open('log.txt', 'a')
f.write(f"{datetime.datetime.now()} - La predicton {'UP' if prediction == 2 else ('DOWN' if prediction == 0 else 'NO_MOVEMENT')} a été émise.\n")
f.close()